In [1]:
import pandas as pd
# pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
import os
import numpy as np
from scipy import stats
from sklearn.neighbors import KNeighborsClassifier

In [2]:
# filepath = './_data/fingerprint'
filepath = './redo15'

filelist = os.listdir(filepath)
filelist

def dfinator(path):
    nmlist = ['time', 'macID','vendor','access_pt','signal','idk','idk_2','freq','station_info','specs']
    filelist = os.listdir(path)
    megadf = pd.DataFrame(columns=nmlist)
    for file_ in filelist:
        df = pd.read_csv(os.path.join(path,file_)  , delimiter='\t', names=nmlist)
        df['place'] = str(file_).strip(".txt")
        # df.time = pd.to_datetime(df.time)
        megadf=megadf.append(df, ignore_index=True)
    return megadf

mega = dfinator(filepath)
mega.to_csv("./_data/megadf.csv")    

eduroam_df = mega[mega['access_pt']=='eduroam'].copy()
eduroam_df.reset_index(inplace=True)
eduroam_df.drop('index', axis=1 , inplace=True)
print("no of unique eduroam mac id:\t",len(eduroam_df.macID.unique()))
print(f"unique places are {eduroam_df.place.unique()}")
eduroam_df.head(15)


no of unique eduroam mac id:	 31
unique places are ['2_031' '2_032' '2_033' '2_034']


,time,macID,vendor,access_pt,signal,idk,idk_2,freq,station_info,specs,place
0,19/5/2021 4:50:27 μμ,84:3D:C6:C7:EF:20,NaN,eduroam,-81,35,NaN,Ch 6 [2.437 GHz],0 Stations,Wi-Fi [Intel(R) Wireless-AC 9462],2_031
1,19/5/2021 4:50:27 μμ,84:3D:C6:B4:E3:BF,NaN,eduroam,-82,33,NaN,Ch 116 [5.580 GHz],0 Stations,Wi-Fi [Intel(R) Wireless-AC 9462],2_031
2,19/5/2021 4:50:27 μμ,84:3D:C6:B9:2B:10,NaN,eduroam,-80,38,NaN,Ch 1 [2.412 GHz],0 Stations,Wi-Fi [Intel(R) Wireless-AC 9462],2_031
3,19/5/2021 4:50:27 μμ,84:3D:C6:B9:2B:1F,NaN,eduroam,-79,40,NaN,Ch 48 [5.240 GHz],0 Stations,Wi-Fi [Intel(R) Wireless-AC 9462],2_031
4,19/5/2021 4:50:27 μμ,00:3A:7D:27:B3:9F,NaN,eduroam,-83,31,NaN,Ch 36 [5.180 GHz],1 Station,Wi-Fi [Intel(R) Wireless-AC 9462],2_031
5,19/5/2021 4:50:27 μμ,00:3A:7D:1E:DC:FF,NaN,eduroam,-68,72,NaN,Ch 132 [5.660 GHz],1 Station,Wi-Fi [Intel(R) Wireless-AC 9462],2_031
6,19/5/2021 4:50:27 μμ,00:3A:7D:34:79:FF,NaN,eduroam,-77,46,NaN,Ch 100 [5.500 GHz],0 Stations,Wi-Fi [Intel(R) Wireless-AC 9462],2_031
7,19/5/2021 4:50:27 μμ,84:3D:C6:C7:ED:1F,NaN,eduroam,-72,62,NaN,Ch 100 [5.500 GHz],0 Stations,Wi-Fi [Intel(R) Wireless-AC 9462],2_031
8,19/5/2021 4:50:27 μμ,00:3A:7D:21:42:EF,NaN,eduroam,-49,89,NaN,Ch 64 [5.320 GHz],13 Stations,Wi-Fi [Intel(R) Wireless-AC 9462],2_031
9,19/5/2021 4:50:27 μμ,00:3A:7D:21:45:8F,NaN,eduroam,-79,40,NaN,Ch 60 [5.300 GHz],5 Stations,Wi-Fi [Intel(R) Wireless-AC 9462],2_031


In [3]:
path30 = './_data/30sec/'
thirty = dfinator(path30)
thirty = thirty[thirty['access_pt']=='eduroam'].drop(['time','vendor','access_pt','idk','idk_2','freq','station_info','specs'], axis=1)


thirty['uniqueID30'] = thirty['macID']+'__'+ thirty['place']
thirty.drop(['macID','place'], axis=1, inplace=True)
thirty.set_index('uniqueID30', inplace=True)
thirty.signal = pd.to_numeric(thirty.signal)
print(len(thirty.index.unique()))

# thirty = thirty.groupby('uniqueID30').agg(lambda x: x.mean())
thirty.reset_index(inplace=True)
thirty['place'] = thirty['uniqueID30'].apply(lambda x: x.split('__')[-1])
thirty['mac'] = thirty['uniqueID30'].apply(lambda x: x.split('__')[0])
print(thirty.place.unique())
thirty_24 = thirty[thirty['place']=='24_aisle']
thirty_24.head()


96
['21_win' '22_aisle' '24_aisle']


,uniqueID30,signal,place,mac
871,00:3A:7D:21:45:8F__24_aisle,-67,24_aisle,00:3A:7D:21:45:8F
872,00:3A:7D:1E:DC:F0__24_aisle,-50,24_aisle,00:3A:7D:1E:DC:F0
873,84:3D:C6:AD:B3:D0__24_aisle,-82,24_aisle,84:3D:C6:AD:B3:D0
874,00:3A:7D:34:77:DF__24_aisle,-70,24_aisle,00:3A:7D:34:77:DF
875,84:3D:C6:C7:F1:EF__24_aisle,-89,24_aisle,84:3D:C6:C7:F1:EF


In [16]:
mldf = eduroam_df.drop(['time','vendor','access_pt','idk','idk_2','freq','station_info','specs'], axis=1)
mldf['uniqueID'] = mldf['macID']+'__'+mldf['place']

a = mldf.macID.unique()
mldf['macno'] = mldf.macID.apply(lambda x: np.where(a==x))#[0][0])
mldf
type(np.where(a=="00:3A:7D:21:42:EF"))



tuple

In [44]:

x_train = mldf[['signal' , 'macno']]
y_train = pd.Series(mldf.place)
# print(x_train)
# print(y_train)
# mldf

def indexer(a,x):
    ind = np.where(a==x)[0]
    if len(ind)!=0:
        return ind[0]

x_test = thirty_24[['signal' , 'mac']].copy()
y_test = pd.Series(thirty_24.place)
x_test['macnot'] = x_test.mac.apply(lambda x: indexer(a,x))
x_test.drop(['mac'], axis=1, inplace=True)
x_test.dropna(inplace=True)
x_test.head()

knn = KNeighborsClassifier(n_neighbors = 7)
knn.fit(x_train , y_train)
pred = knn.predict(x_test)

print(f"Predicted room is {stats.mode(pred)[0][0]}, actual room is {str(y_test.iloc[0])}. Prediction done with a score of {knn.score(x_test, y_test[:x_test.shape[0]])}")

Predicted room is 24_aisle, actual room is 24_aisle. Prediction done with a score of 0.40615384615384614


In [48]:
stats.mode(pred)

ModeResult(mode=array(['24_aisle'], dtype=object), count=array([132]))